In [1]:
### run when it starts
pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import pipeline
import pandas as pd
import sqlalchemy
from dotenv import dotenv_values
from dotenv import load_dotenv
import sql_functions as sf
from sql_functions import get_sql_config, get_data, get_dataframe, get_engine

In [ ]:
### need tensorflow > check if its latest
pip install --upgrade tensorflow

In [4]:
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-699.3589, shape=(), dtype=float32)


### 1. Sentiment Analysis

We tested several models for doing sentiment analysis of the Yelp reviews, in the end we decided to move forward with the one pipeline below. 

In [6]:
#  select the pipeline: https://huggingface.co/mrcaelumn/yelp_restaurant_review_sentiment_analysis

sentiment_pipeline = pipeline(model="mrcaelumn/yelp_restaurant_review_sentiment_analysis")

In [ ]:
# query the original review data from server
review_pa = sf.get_dataframe('SELECT * FROM hh_analytics_23_3.yelp_review_pa') 

# 4 columns in the result df
sentiment_result = pd.DataFrame(columns=['business_id', 'review_id', 'text', 'result'])

# loop each row(review)
for index, row in review_pa.iterrows():
    try:
        # get each review info
        review_id = row['review_id']
        business_id = row['business_id']
        text = row['text']

        # processing by the model
        result = sentiment_pipeline(text)

        row_result = pd.DataFrame({
            'business_id': [business_id],
            'review_id': [review_id],
            'text': [text],
            'result': [result],
        })

        sentiment_result = pd.concat([sentiment_result, row_result], ignore_index=True)
# the model can not processing long text (dont know how long), but this will skip the error
    except Exception as e:
        
        print(f"Error processing text: {text}. Error: {e}")
        continue

# write the result in the csv locally
sentiment_result.to_csv('review_pa_sentiment.csv', index=False)


### 2. Merge review_sentiment with business_df

In [1]:
# import required packages and functions from sql_functions
import json
import pandas as pd
import matplotlib.pyplot as plt
# Set style for plots
plt.style.use('fivethirtyeight')
# import nacessary packages for pSql
import sqlalchemy
from dotenv import dotenv_values
from dotenv import load_dotenv
import psycopg2
import sql_functions as sf
from sql_functions import get_sql_config, get_data, get_dataframe, get_engine

In [2]:
# set variable for pSql schema and table
schema = 'hh_analytics_23_3'
review_table = 'yelp_review_pa'
business_table = 'yelp_business_all'

In [3]:
# Read the rows from the Review table as review_df. Approx runtime 2m 48s
review_df = sf.get_dataframe(f'SELECT * FROM {schema}.{review_table}')

In [4]:
# Read business table from psql
business = sf.get_dataframe(f'SELECT * FROM {schema}.{business_table}')

In [12]:
# read csv analysis for sentiment on review table
review_sentiment = pd.read_csv('data/review_pa_sentiment.csv')
review_sentiment.shape

(1184897, 4)

#### Split by sentiment labels

In [13]:
# Separate labels into different columns. Approx runtime 1m 27s
for index, row in review_sentiment.iterrows():
    result_dict = json.loads(review_sentiment['result'][index].replace("'", '"'))
    result_series = pd.Series(result_dict[0])
    label = result_series['label']
    score = result_series['score']
    review_sentiment.at[index,label] = score

print(review_sentiment)

                    business_id               review_id  \
0        zRjUMHQJ5gAmFhcXZtLacA  MbC7LA9y1rcVxtVKF3RaSg   
1        VHA-b1NH4tIZB_IVEf_pmA  sSQKHpFs8tSXl4Lv34zQgQ   
2        cPDX2Y4H0fks-Qj5tfBduA  _B5qqp5srqGFiHio1Wl-FA   
3        UljbSHv00PKS6YUnnCiSZA  PIHnj9k4ow29r7CVwcjfYA   
4        c1okcAnrDc9jIy3DbOUbqQ  qShqGHbUTD48-7CMNmOZWg   
...                         ...                     ...   
1184892  iSSErxB_4XLkrKU0CotYMg  y9KnJsni5eqZrPYOz9RKZQ   
1184893  qcguEeAMP0XwFLYqhwX2hg  kKg785BzBgMl2ObxQZ244g   
1184894  9c7MUiE6VI8NesjPdj5FkA  Hx20Q_Vf9f4TCmx5tCMkzg   
1184895  jLoQf_d2DmBGTlccjkpEIA  Z8BSakOHx2C6A80R4tLbAA   
1184896  j-qtdD55OLfSqfsWuQTDJg  -ASFj7UT6JXqs4pkWi5p2Q   

                                                      text  \
0        OK. I'll settle this issue and we can be done ...   
1        I have to say that Minado is a above the avera...   
2        Love the little window concept. Very cute pink...   
3        We frequent this place often becau

#### Rename header and drop result file

In [14]:
# Rename headers by the name of label in dataframe of review_sentiment
review_sentiment.rename(columns={'LABEL_2': 'positive', 'LABEL_1': 'neutral','LABEL_0': 'negative' }, inplace=True)
# drop results column
review_sentiment.drop(['result'], axis=1, inplace=True)
review_sentiment

,business_id,review_id,text,positive,neutral,negative
0,zRjUMHQJ5gAmFhcXZtLacA,MbC7LA9y1rcVxtVKF3RaSg,OK. I'll settle this issue and we can be done ...,0.935561,NaN,NaN
1,VHA-b1NH4tIZB_IVEf_pmA,sSQKHpFs8tSXl4Lv34zQgQ,I have to say that Minado is a above the avera...,NaN,0.793211,NaN
2,cPDX2Y4H0fks-Qj5tfBduA,_B5qqp5srqGFiHio1Wl-FA,Love the little window concept. Very cute pink...,0.997141,NaN,NaN
3,UljbSHv00PKS6YUnnCiSZA,PIHnj9k4ow29r7CVwcjfYA,We frequent this place often because the food ...,0.995334,NaN,NaN
4,c1okcAnrDc9jIy3DbOUbqQ,qShqGHbUTD48-7CMNmOZWg,"Ate here on 2 different occasions, and in both...",0.995351,NaN,NaN
...,...,...,...,...,...,...
1184892,iSSErxB_4XLkrKU0CotYMg,y9KnJsni5eqZrPYOz9RKZQ,I've had both Pat's and Geno's and Jim's beats...,0.921866,NaN,NaN
1184893,qcguEeAMP0XwFLYqhwX2hg,kKg785BzBgMl2ObxQZ244g,"ADORABLE inside, albeit a bit bare still, supe...",0.972836,NaN,NaN
1184894,9c7MUiE6VI8NesjPdj5FkA,Hx20Q_Vf9f4TCmx5tCMkzg,This place has never been fast but man it seem...,NaN,NaN,0.93691
1184895,jLoQf_d2DmBGTlccjkpEIA,Z8BSakOHx2C6A80R4tLbAA,"m mm mm'm, mmm, mm'm, mm m mmm mm m! And that...",0.996134,NaN,NaN


#### Now join review stars and user_id into sentiments then business stars and save in CSV

In [15]:
# Inner join review_df in review_sentiments columns stars on column review_id
review_all = review_sentiment.merge(review_df[['review_id', 'user_id', 'stars']], how='inner', on='review_id')
review_all.head()

,business_id,review_id,text,positive,neutral,negative,user_id,stars
0,zRjUMHQJ5gAmFhcXZtLacA,MbC7LA9y1rcVxtVKF3RaSg,OK. I'll settle this issue and we can be done ...,0.935561,NaN,NaN,mdHrElt_ZxLbG5QLpFSYPQ,3
1,VHA-b1NH4tIZB_IVEf_pmA,sSQKHpFs8tSXl4Lv34zQgQ,I have to say that Minado is a above the avera...,NaN,0.793211,NaN,kZOHcAsSBIAoXn7cYbvveA,3
2,cPDX2Y4H0fks-Qj5tfBduA,_B5qqp5srqGFiHio1Wl-FA,Love the little window concept. Very cute pink...,0.997141,NaN,NaN,3G9zwCpr-pxQhO2D8-wvxQ,4
3,UljbSHv00PKS6YUnnCiSZA,PIHnj9k4ow29r7CVwcjfYA,We frequent this place often because the food ...,0.995334,NaN,NaN,cARxOd_5yKCgsCbUZ5ED4Q,4
4,c1okcAnrDc9jIy3DbOUbqQ,qShqGHbUTD48-7CMNmOZWg,"Ate here on 2 different occasions, and in both...",0.995351,NaN,NaN,Xm9AqniqmfmOqynmIOsNWg,5


In [16]:
# Inner join business df to review df
review_all = review_all.merge(business[['business_id', 'stars']], how='inner', on='business_id')
# Rename stars while one is review stars and another is business stars
review_all.rename(columns={'stars_x': 'review_stars', 'stars_y':'stars'}, inplace=True)
# Writing to csv
review_all.to_csv('data/review_all.csv', index=False)
review_all.head()

,business_id,review_id,text,positive,neutral,negative,user_id,review_stars,stars
0,zRjUMHQJ5gAmFhcXZtLacA,MbC7LA9y1rcVxtVKF3RaSg,OK. I'll settle this issue and we can be done ...,0.935561,NaN,NaN,mdHrElt_ZxLbG5QLpFSYPQ,3,4.0
1,zRjUMHQJ5gAmFhcXZtLacA,7YLYYNfhZolc9f6VteOYrw,My review of this place is a little mixed and ...,NaN,0.537727,NaN,lDmj-ZHRsh1b_H82vZ_wCg,3,4.0
2,zRjUMHQJ5gAmFhcXZtLacA,NE78S_6nYH9a2e80yKaB4w,Still the best but still over priced. I don't ...,0.742791,NaN,NaN,GfI-d9mQePFA2PvhAd4WGQ,4,4.0
3,zRjUMHQJ5gAmFhcXZtLacA,Z2vIhXNgRPBCkw39u8aV1w,This restaurant might be able to fool the non-...,0.814609,NaN,NaN,VIQl89t4xTe9KFq7I5wBHg,1,4.0
4,zRjUMHQJ5gAmFhcXZtLacA,eOdqvTnJ2wJHv7hztli2RQ,"Friendly staff, clean establishment, and yummy...",0.996823,NaN,NaN,ObATy40FeJ8hMv1tAhcPeA,5,4.0
